In [1]:
import pandas as pd
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Loads labels
df = pd.read_csv('../data/seedling_labels_with_features_and_predictions.csv')

#df = df.drop(columns=['Pos','average_expert'])
#df = df.iloc[:, -8:]
decisions = df.loc[:,['Expert 1','Expert 2','Expert 3', 'Expert 4']]
df

,Expert 1,Expert 2,Expert 3,Expert 4,color_cam_path,side_cam_path,Rfid,Pos,average_expert,expert_binary,...,plant_height_side_view_lr_probability,plant_height_side_view_lr_binary,plant_area_side_view_lr_probability,plant_area_side_view_lr_binary,plant_width_top_view_lr_probability,plant_width_top_view_lr_binary,plant_height_top_view_lr_probability,plant_height_top_view_lr_binary,plant_area_top_view_lr_probability,plant_area_top_view_lr_binary
0,4,4,4,4,data\C:\Users\Philip Salomons\OneDrive\Documen...,data\C:\Users\Philip Salomons\OneDrive\Documen...,A1,Plant 0000,4.00,0,...,0.099614,0,0.608102,1,0.211777,0,0.134455,0,0.663127,1
1,1,1,1,1,data\C:\Users\Philip Salomons\OneDrive\Documen...,data\C:\Users\Philip Salomons\OneDrive\Documen...,A1,Plant 0001,1.00,1,...,0.766689,1,0.815682,1,0.943884,1,0.845458,1,0.958443,1
2,1,1,1,1,data\C:\Users\Philip Salomons\OneDrive\Documen...,data\C:\Users\Philip Salomons\OneDrive\Documen...,A1,Plant 0002,1.00,1,...,0.884680,1,0.837040,1,0.748042,1,0.973192,1,0.960792,1
3,4,4,3,3,data\C:\Users\Philip Salomons\OneDrive\Documen...,data\C:\Users\Philip Salomons\OneDrive\Documen...,A1,Plant 0003,3.50,0,...,0.105179,0,0.609529,1,0.252940,0,0.132932,0,0.701503,1
4,3,1,1,1,data\C:\Users\Philip Salomons\OneDrive\Documen...,data\C:\Users\Philip Salomons\OneDrive\Documen...,A1,Plant 0004,1.50,1,...,0.950068,1,0.896045,1,0.980442,1,0.819863,1,0.977412,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,1,1,1,1,data\C:\Users\Philip Salomons\OneDrive\Documen...,data\C:\Users\Philip Salomons\OneDrive\Documen...,B4,Plant 0122,1.00,1,...,0.990059,1,0.969487,1,0.907459,1,0.478031,0,0.741524,1
990,1,1,1,1,data\C:\Users\Philip Salomons\OneDrive\Documen...,data\C:\Users\Philip Salomons\OneDrive\Documen...,B4,Plant 0123,1.00,1,...,0.997249,1,0.935264,1,0.968125,1,0.940695,1,0.778464,1
991,1,1,1,1,data\C:\Users\Philip Salomons\OneDrive\Documen...,data\C:\Users\Philip Salomons\OneDrive\Documen...,B4,Plant 0124,1.00,1,...,0.966747,1,0.927234,1,0.891207,1,0.949539,1,0.915575,1
992,2,3,3,3,data\C:\Users\Philip Salomons\OneDrive\Documen...,data\C:\Users\Philip Salomons\OneDrive\Documen...,B4,Plant 0125,2.75,0,...,0.704065,1,0.733834,1,0.799582,1,0.674121,1,0.744187,1


In [2]:
# Define a function to convert the values to binary
def convert_to_binary(x):
    if x in [1, 2]:
        return 1
    else:
        return 0

# Apply the function to each element in the dataframe
decisions = decisions.applymap(convert_to_binary)
features = df.iloc[:,10:16]
voting = pd.concat([features,decisions],axis = 1)
voting.head()

,plant_width_side_view,plant_height_side_view,plant_area_side_view,plant_width_top_view,plant_height_top_view,plant_area_top_view,Expert 1,Expert 2,Expert 3,Expert 4
0,120,239,8577,263,277,18368,0,0,0,0
1,591,407,29037,585,548,85112,1,1,1,1
2,388,449,31946,450,692,86757,1,1,1,1
3,92,242,8694,281,276,23174,0,0,0,0
4,582,494,42052,670,534,102178,0,1,1,1


In [3]:
##MAJORITY VOTING

# define ground truth as the expert majority vote with 75%+ agreement
true_labels = df["expert_binary"]


In [4]:
from scipy.stats import beta
# Define the fusion function
def bayesian_fusion(expert_decisions):
    # Compute the beta distribution parameters based on expert decisions
    alpha = expert_decisions.sum()
    beta_ = expert_decisions.size - alpha
    
    # Compute the posterior probability distribution
    p = beta.cdf(0.5, alpha, beta_)
    
    return p

# Apply the fusion function to the dataset
voting['bayes_consensus'] = decisions.apply(bayesian_fusion, axis=1)

# Define the decision threshold
threshold = 0.5

# Make final classification decisions based on the threshold
voting['bayes_consensus'] = (voting['bayes_consensus'] > threshold).astype(int)
print(voting.head())

   plant_width_side_view  plant_height_side_view  plant_area_side_view  \
0                    120                     239                  8577   
1                    591                     407                 29037   
2                    388                     449                 31946   
3                     92                     242                  8694   
4                    582                     494                 42052   

   plant_width_top_view  plant_height_top_view  plant_area_top_view  Expert 1  \
0                   263                    277                18368         0   
1                   585                    548                85112         1   
2                   450                    692                86757         1   
3                   281                    276                23174         0   
4                   670                    534               102178         0   

   Expert 2  Expert 3  Expert 4  bayes_consensus  
0         0      

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from scipy.stats import mode
from scipy.stats import bayes_mvs

X = voting.iloc[:, 0:7]
y = true_labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=57)


In [6]:
# Train different classifiers
clf1 = RandomForestClassifier(random_state=57)
clf2 = AdaBoostClassifier(random_state=57)
clf3 = LogisticRegression(random_state=57)
clf4 = GaussianNB()

clf1.fit(X_train, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train, y_train)
clf4.fit(X_train, y_train)


GaussianNB()

In [7]:
y_pred1 = clf1.predict(X_test)
y_pred2 = clf2.predict(X_test)
y_pred3 = clf3.predict(X_test)
y_pred4 = clf4.predict(X_test)

y_preds = np.vstack((y_pred1, y_pred2, y_pred3, y_pred4))


In [8]:
from sklearn.metrics import confusion_matrix
conf1 = confusion_matrix(y_test, y_pred1, normalize = 'true')
conf2 = confusion_matrix(y_test, y_pred2, normalize = 'true')
conf3 = confusion_matrix(y_test, y_pred3, normalize = 'true')
conf4 = confusion_matrix(y_test, y_pred4, normalize = 'true')

In [12]:
from scipy.stats import bayes_mvs

def decision_fusion(y_preds, method='hard'):
    if method == 'hard':
        return mode(y_preds, axis=0)[0][0]
    elif method == 'soft':
        return np.round(np.mean(y_preds, axis=0))
    elif method == 'bayesian':
        return 0
    else:
        raise ValueError("Invalid method")


In [13]:
y_pred_hard = decision_fusion(y_preds, method='hard')
y_pred_soft = decision_fusion(y_preds, method='soft')
y_pred_bayesian = decision_fusion(y_preds, method='bayesian')


In [15]:
print("Hard Voting Accuracy:", accuracy_score(y_test, y_pred_hard))
print("Soft Voting Accuracy:", accuracy_score(y_test, y_pred_soft))
#print("Bayesian Consensus Accuracy:", accuracy_score(y_test, y_pred_bayesian))


Hard Voting Accuracy: 0.9296482412060302
Soft Voting Accuracy: 0.9296482412060302
